In [378]:
run main_imports.py

In [13]:
import gensim

In [34]:
pd.options.plotting.backend = 'plotly'

In [2]:
df = pd.read_csv('music.csv')

In [64]:
df.head()

,Written_by,Artist,Album,Song,Song_initial,Year,Producer,Lyrics,Y,Splitted_Lyrics
0,NaN,Red Hot Chili Peppers,Californication,Around the World,Around the World,1999-09-14,Rick Rubin,All around the world we could make time Rompin...,1999.0,"[All around the world we could make time , Rom..."
1,NaN,Red Hot Chili Peppers,Californication,Parallel Universe,Parallel Universe,1999-06-08,Rick Rubin,Deep inside of a parallel universe It s gettin...,1999.0,"[Deep inside of a parallel universe , It s get..."
2,NaN,Red Hot Chili Peppers,Californication,Scar Tissue,Scar Tissue,1999-05-25,Rick Rubin,Scar tissue that I wish you saw Sarcastic mist...,1999.0,"[Scar tissue that , I wish you saw , Sarcastic..."
3,NaN,Red Hot Chili Peppers,Californication,Otherside,Otherside,1999-06-08,Rick Rubin,How long how long will I slide Separate my sid...,1999.0,"[How long how long will , I slide , Separate m..."
4,NaN,Red Hot Chili Peppers,Californication,Get on Top,Get on Top,1999-06-08,Rick Rubin,Hit me come get me I bite but she bit me Go-ri...,1999.0,"[Hit me come get me , I bite but she bit me , ..."


In [364]:
df=df.dropna(subset=['Lyrics'])

In [16]:
df['Splitted_Lyrics']=df['Lyrics'].progress_apply(lambda x: re.findall('[A-Z][^A-Z]*',str(x)))

100%|██████████| 77101/77101 [00:07<00:00, 10204.38it/s]


In [120]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 77101 entries, 0 to 77100
Data columns (total 10 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Written_by       37889 non-null  object 
 1   Artist           77101 non-null  object 
 2   Album            70804 non-null  object 
 3   Song             77101 non-null  object 
 4   Song_initial     76794 non-null  object 
 5   Year             50693 non-null  object 
 6   Producer         52744 non-null  object 
 7   Lyrics           75823 non-null  object 
 8   Y                50693 non-null  float64
 9   Splitted_Lyrics  77101 non-null  object 
dtypes: float64(1), object(9)
memory usage: 5.9+ MB


In [121]:
df['Year'] = df['Year'].progress_apply(pd.to_datetime)

100%|██████████| 77101/77101 [00:06<00:00, 12003.31it/s]


In [168]:
df['Quarter']=df['Year'].progress_apply(lambda x: x.quarter)
df['Month'] = df['Year'].progress_apply(lambda x: x.month)
df['Weekday'] = df['Year'].progress_apply(lambda x: x.dayofweek)

100%|██████████| 77101/77101 [00:00<00:00, 320504.97it/s]


In [79]:
df[['Artist','Album','Song','Y']].drop_duplicates()['Artist'].value_counts()[:25].plot(kind='bar',title = 'Top Productive Artists ;)')

In [186]:
df['Song_len'] = df['Lyrics'].progress_apply(lambda x: len(str(x).split()))

100%|██████████| 77101/77101 [00:05<00:00, 13717.52it/s]


In [213]:
gb_artist_song_len=df.groupby('Artist')['Song_len'].transform('mean')
gb_artist_song_len

0           272.561576
1           272.561576
2           272.561576
3           272.561576
4           272.561576
             ...      
77096      7395.500000
77097       375.000000
77098       922.000000
77099       853.200000
77100    107855.333333
Name: Song_len, Length: 77101, dtype: float64

In [228]:
df= df[df['Song_len']<np.quantile(gb_artist_song_len,0.95)]

In [256]:
df.groupby('Artist')['Song'].transform('count').plot.hist()

In [257]:
df=df[(df.groupby('Artist')['Song'].transform('count')>19)]

In [258]:
df[['Artist','Y']].Y.value_counts().plot(kind='bar')

In [260]:
df.groupby(['Y','Artist'])['Song'].agg('count').reset_index().plot(kind='hist',x='Y',y='Song',title='Song Count by year')

In [261]:
df.groupby(['Quarter','Month'])['Song'].count().reset_index().plot.bar(x='Quarter',y='Song',color='Month',title='Song count by Quart\Month')

In [232]:
df.groupby(['Month','Weekday'])['Song'].count().reset_index().plot.bar(x='Month',y='Song',color='Weekday',title='Song count by Month\DayofWeek')

In [403]:
dv = gensim.models.Doc2Vec(epochs=10,min_count=2,vector_size=300)

In [404]:
tagged = [gensim.models.doc2vec.TaggedDocument(Lyrics,[Artist]) for Lyrics,Artist in tqdm(df[['Lyrics','Artist']].values)]

100%|██████████| 64623/64623 [00:00<00:00, 483429.90it/s]


In [405]:
dv.build_vocab(tagged)

In [401]:
for i in tqdm(range(10)):
    dv.train(tagged,total_examples=dv.corpus_count,epochs=1)

100%|██████████| 10/10 [03:03<00:00, 18.31s/it]


In [406]:
dv.train(tagged,total_examples=dv.corpus_count,epochs=10)

In [408]:
dv.docvecs.most_similar('Queen')

[('Level 42', 0.4698446989059448),
 ('Frank Sinatra', 0.43822425603866577),
 ('Crosby, Stills, Nash & Young', 0.4380917549133301),
 ('Nat King Cole', 0.4090685248374939),
 ('Blood, Sweat & Tears', 0.3991771936416626),
 ('Chance the Rapper', 0.3752414882183075),
 ('Country Genius', 0.3715498447418213),
 ('System of a Down', 0.3683876395225525),
 ('Earth, Wind & Fire', 0.35847607254981995),
 ('Steve Winwood', 0.358460009098053)]

In [412]:
df.Artist.value_counts()

Elvis Presley     532
Glee Cast         428
Frank Sinatra     409
Johnny Cash       385
Bob Dylan         370
                 ... 
Cream Soda         15
Flying Lotus       13
​deadmau5          11
Randy Newman        3
The Three Suns      1
Name: Artist, Length: 878, dtype: int64

In [423]:
df['Vectors'] = df['Artist'].progress_apply(lambda x: dv.docvecs[i])

100%|██████████| 64623/64623 [00:00<00:00, 498432.35it/s]


In [453]:
df_tsne=manifold.TSNE(n_components=2,random_state=1,perplexity=150).fit_transform(dv.docvecs.vectors_docs)

In [454]:
df_tsne=pd.DataFrame(df_tsne,columns=['x','y'])

In [456]:
df_tsne['Artist'] = dv.docvecs.doctags.keys()

In [462]:
gb_len=df.groupby('Artist')['Song_len'].mean().reset_index()

In [465]:
df_tsne=gb_len.merge(df_tsne,on='Artist')

In [467]:
df_tsne.query('Song_len<800').plot.scatter(x='x',y='y',hover_name='Artist',color='Song_len')